In [ ]:
# Stažení brain2vec repozitáře
!git clone https://huggingface.co/radiata-ai/brain2vec
%cd brain2vec

# Instalace git-lfs a stažení modelových vah
!apt-get update
!apt-get install -y git-lfs
!git lfs install
!git lfs pull

# Instalace potřebných knihoven
!pip install -r requirements.txt
!pip install SimpleITK nibabel

# Vytvoření potřebných adresářů
!mkdir -p ae_cache ae_output

In [ ]:
%%writefile /kaggle/working/brain2vec/preprocess_data.py

from itertools import product
import os
import pandas as pd
import numpy as np
import SimpleITK as sitk
import nibabel as nib
from glob import glob
import re
from scipy.ndimage import gaussian_filter, binary_dilation

def convert_mha_to_nifti(mha_file, output_dir):
    """
    Převede jeden MHA soubor na NIfTI formát.
    
    Args:
        mha_file: Cesta k MHA souboru
        output_dir: Výstupní adresář pro uložení NIfTI souborů
        
    Returns:
        Cesta k vytvořenému NIfTI souboru
    """
    # Načtení MHA souboru
    img = sitk.ReadImage(mha_file)
    
    # Převod na numpy - bez interpolace
    data = sitk.GetArrayFromImage(img)
    
    # Normalizace dat
    data = np.clip(data, 0, None)
    if data.max() > 0:
        data = data / data.max()
    
    # Pro léze zajistíme binární masku
    if 'lesion' in mha_file.lower():
        data = (data > 0.1).astype(np.float32)
    
    # Vytvoření afinní matice pro zachování orientace
    direction = np.array(img.GetDirection()).reshape(3, 3)
    spacing = np.array(img.GetSpacing())
    origin = np.array(img.GetOrigin())
    
    affine = np.eye(4)
    affine[:3, :3] = direction * np.expand_dims(spacing, 1)
    affine[:3, 3] = origin
    
    # Vytvoření a uložení NIfTI
    nifti_img = nib.Nifti1Image(data, affine)
    base_name = os.path.basename(mha_file).replace('.mha', '.nii.gz')
    output_path = os.path.join(output_dir, base_name)
    nib.save(nifti_img, output_path)
    
    return output_path

# Definujeme cesty k datům v Kaggle
pseudo_healthy_dir = '/kaggle/input/pseudohealthy-masked/masked_pseudohealthy'
adc_dir = '/kaggle/input/bonbid-2023-train/BONBID2023_Train/1ADC_ss'
label_dir = '/kaggle/input/bonbid-2023-train/BONBID2023_Train/3LABEL'
output_dir = '/kaggle/working/nifti_data'

# Vytvoření výstupního adresáře
os.makedirs(output_dir, exist_ok=True)

# Počet souborů v každém adresáři
ph_count = len(glob(os.path.join(pseudo_healthy_dir, "*.mha")))
adc_count = len(glob(os.path.join(adc_dir, "*.mha")))
label_count = len(glob(os.path.join(label_dir, "*.mha")))

print(f"\nPočty souborů: Pseudozdravé: {ph_count}, ADC: {adc_count}, Léze: {label_count}")

# Zkusíme přímější způsob párování - předpokládáme, že soubory mají konzistentní části názvů
matched_data = []

# Procházíme pseudozdravé soubory, jelikož jich máme nejméně
for ph_file in sorted(glob(os.path.join(pseudo_healthy_dir, "*.mha"))):
    ph_basename = os.path.basename(ph_file)
    # Tady upravte regulární výraz podle skutečného formátu vašich souborů
    match = re.search(r'(MGHNICU_\d+-VISIT_\d+)', ph_basename)
    
    if match:
        pattern_id = match.group(1)
        # Hledáme odpovídající ADC a léze soubory
        adc_pattern = os.path.join(adc_dir, f"{pattern_id}*ADC_ss.mha")
        lesion_pattern = os.path.join(label_dir, f"{pattern_id}*lesion.mha")
        
        adc_matches = glob(adc_pattern)
        lesion_matches = glob(lesion_pattern)
        
        if adc_matches and lesion_matches:
            # Konverze MHA na NIfTI
            ph_nifti = convert_mha_to_nifti(ph_file, output_dir)
            adc_nifti = convert_mha_to_nifti(adc_matches[0], output_dir)
            lesion_nifti = convert_mha_to_nifti(lesion_matches[0], output_dir)
            
            matched_data.append({
                'subject_id': pattern_id,
                'pseudo_healthy': ph_nifti,
                'adc': adc_nifti,
                'lesion': lesion_nifti
            })
            print(f"Nalezena shoda pro: {pattern_id}")
        else:
            print(f"Nenalezeny odpovídající soubory pro: {pattern_id}")
            if not adc_matches:
                print(f"  Žádné ADC soubory pro vzor: {adc_pattern}")
            if not lesion_matches:
                print(f"  Žádné léze soubory pro vzor: {lesion_pattern}")

# Pokud stále nemáme žádné shody, zkusíme flexibilnější párování
if not matched_data:
    # Pro každý pseudozdravý soubor
    for ph_file in sorted(glob(os.path.join(pseudo_healthy_dir, "*.mha"))):
        ph_basename = os.path.basename(ph_file)
        
        # Extrakt základního názvu bez přípony a specifických označení
        base_name = ph_basename.replace('-PSEUDO_HEALTHY.mha', '').replace('_PSEUDO_HEALTHY.mha', '')
        
        # Hledání odpovídajících souborů podle základního názvu
        for adc_file in sorted(glob(os.path.join(adc_dir, "*.mha"))):
            adc_basename = os.path.basename(adc_file)
            
            # Kontrola, zda ADC soubor obsahuje základní název
            if base_name in adc_basename:
                # Hledání odpovídajícího léze souboru
                for lesion_file in sorted(glob(os.path.join(label_dir, "*.mha"))):
                    lesion_basename = os.path.basename(lesion_file)
                    
                    # Kontrola, zda léze soubor obsahuje základní název
                    if base_name in lesion_basename:
                        # Konverze MHA na NIfTI
                        ph_nifti = convert_mha_to_nifti(ph_file, output_dir)
                        adc_nifti = convert_mha_to_nifti(adc_file, output_dir)
                        lesion_nifti = convert_mha_to_nifti(lesion_file, output_dir)
                        
                        matched_data.append({
                            'subject_id': base_name,
                            'pseudo_healthy': ph_nifti,
                            'adc': adc_nifti,
                            'lesion': lesion_nifti
                        })
                        break  # Po nalezení léze přejít na další ADC

# Vytvoření CSV souboru
df = pd.DataFrame(matched_data)
csv_path = '/kaggle/working/brain2vec/inputs.csv'
df.to_csv(csv_path, index=False)
print(f"\nVytvořen CSV soubor s {len(df)} záznamy: {csv_path}")

# Pokud stále nemáme žádné shody, vytvoříme testovací data
if len(df) == 0:
    print("\nNebyly nalezeny žádné shody. Vytvářím umělá testovací data pro testování funkčnosti...")
    
    # Vybereme několik souborů z každého adresáře pro testování
    ph_files = sorted(glob(os.path.join(pseudo_healthy_dir, "*.mha")))[:3]
    adc_files = sorted(glob(os.path.join(adc_dir, "*.mha")))[:3]
    lesion_files = sorted(glob(os.path.join(label_dir, "*.mha")))[:3]
    
    if ph_files and adc_files and lesion_files:
        test_data = []
        
        for i in range(min(len(ph_files), len(adc_files), len(lesion_files))):
            # Konverze MHA na NIfTI
            ph_nifti = convert_mha_to_nifti(ph_files[i], output_dir)
            adc_nifti = convert_mha_to_nifti(adc_files[i], output_dir)
            lesion_nifti = convert_mha_to_nifti(lesion_files[i], output_dir)
            
            test_data.append({
                'subject_id': f'test_subject_{i}',
                'pseudo_healthy': ph_nifti,
                'adc': adc_nifti,
                'lesion': lesion_nifti
            })
        
        # Vytvoření CSV souboru s testovacími daty
        test_df = pd.DataFrame(test_data)
        test_csv_path = '/kaggle/working/brain2vec/inputs.csv'
        test_df.to_csv(test_csv_path, index=False)
        print(f"Vytvořen testovací CSV soubor s {len(test_df)} záznamy: {test_csv_path}")

In [ ]:
%%writefile /kaggle/working/brain2vec/finetune_hie_lesions.py

import os
import argparse
import torch
from itertools import product
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import nibabel as nib
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from generative.networks.nets import AutoencoderKL
import torch.serialization
from numpy.core.multiarray import _reconstruct
from numpy import ndarray, dtype
from monai.data.meta_tensor import MetaTensor
from scipy import ndimage
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from scipy.ndimage import gaussian_filter, binary_dilation
from scipy.ndimage import rotate, shift

# Přidáváme bezpečné globály pro deserializaci objektů z PyTorch
torch.serialization.add_safe_globals([_reconstruct])
torch.serialization.add_safe_globals([MetaTensor])
torch.serialization.add_safe_globals([ndarray])
torch.serialization.add_safe_globals([dtype])

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class VGG19FeatureExtractor(nn.Module):
    def __init__(self, feature_layers=None, use_cuda=True):
        super(VGG19FeatureExtractor, self).__init__()
        if feature_layers is None:
            feature_layers = [2, 7, 12, 21, 30]  # Conv1_2, Conv2_2, Conv3_2, Conv4_2, Conv5_2
        
        self.feature_layers = feature_layers
        
        # Načtení předtrénované VGG19
        vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
        self.vgg_features = vgg.features
        
        # Zmrazení vah
        for param in self.vgg_features.parameters():
            param.requires_grad = False
            
        # Převod na CUDA, pokud je dostupné
        self.device = torch.device('cuda' if use_cuda and torch.cuda.is_available() else 'cpu')
        self.vgg_features = self.vgg_features.to(self.device)
        
    def forward(self, x, slice_idx=None):
        """
        Args:
            x: Vstupní tensor tvaru [B, C, D, H, W] nebo [B, C, H, W]
            slice_idx: Index řezu, pokud je vstup 3D. Pokud None, použije se prostřední řez.
        """
        # Převeďme na 2D, pokud je vstup 3D
        if x.dim() == 5:  # [B, C, D, H, W]
            if slice_idx is None:
                # Použití prostředního řezu
                slice_idx = x.shape[2] // 2
            x = x[:, :, slice_idx, :, :]  # [B, C, H, W]
        
        # VGG očekává 3 kanály (RGB)
        if x.shape[1] == 1:
            x = x.repeat(1, 3, 1, 1)
        
        # Standardizace vstupu do VGG
        mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(x.device)
        std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(x.device)
        x = (x - mean) / std
        
        features = []
        for i, layer in enumerate(self.vgg_features):
            x = layer(x)
            if i in self.feature_layers:
                features.append(x)
                
        return features

class PerceptualLoss(nn.Module):
    def __init__(self, weights=None, use_cuda=True):
        super(PerceptualLoss, self).__init__()
        if weights is None:
            # Váhy pro různé vrstvy - můžete je upravit podle potřeby
            weights = [1.0/32, 1.0/16, 1.0/8, 1.0/4, 1.0]
        
        self.weights = weights
        self.extractor = VGG19FeatureExtractor(use_cuda=use_cuda)
        self.mse = nn.MSELoss()
        
    def forward(self, predicted, target, slice_idx=None):
        """
        Args:
            predicted: Predikovaný obraz [B, 1, D, H, W] nebo [B, 1, H, W]
            target: Cílový obraz [B, 1, D, H, W] nebo [B, 1, H, W]
            slice_idx: Index řezu, pokud jsou vstupy 3D. Pokud None, použije se prostřední řez.
        """
        # Převod do rozsahu [0, 1] pokud jsou v jiném rozsahu
        if predicted.max() > 1 or predicted.min() < 0:
            predicted = (predicted - predicted.min()) / (predicted.max() - predicted.min() + 1e-7)
        if target.max() > 1 or target.min() < 0:
            target = (target - target.min()) / (target.max() - target.min() + 1e-7)
        
        pred_features = self.extractor(predicted, slice_idx)
        target_features = self.extractor(target, slice_idx)
        
        loss = 0.0
        for i in range(len(pred_features)):
            loss += self.weights[i] * self.mse(pred_features[i], target_features[i])
            
        return loss

# Adaptér pro 3D data, který aplikuje perceptual loss na několik řezů
class PerceptualLoss3D(nn.Module):
    def __init__(self, num_slices=3, weights=None, use_cuda=True):
        super(PerceptualLoss3D, self).__init__()
        self.perceptual_loss = PerceptualLoss(weights, use_cuda)
        self.num_slices = num_slices
        
    def forward(self, predicted, target):
        """
        Args:
            predicted: Predikovaný 3D obraz [B, 1, D, H, W]
            target: Cílový 3D obraz [B, 1, D, H, W]
        """
        depth = predicted.shape[2]
        if self.num_slices >= depth:
            # Použij všechny řezy, pokud je jich méně než num_slices
            slice_indices = list(range(depth))
        else:
            # Jinak vybírej rovnoměrně rozložené řezy
            slice_indices = [int(i * (depth-1) / (self.num_slices-1)) for i in range(self.num_slices)]
        
        loss = 0.0
        for idx in slice_indices:
            loss += self.perceptual_loss(predicted, target, idx)
            
        return loss / len(slice_indices)

class BrainLesionAugmentation:
    def __init__(self, 
                 rotation_range=(-10, 10),     # rozsah náhodných rotací ve stupních
                 flip_probability=0.5,         # pravděpodobnost překlápění
                 intensity_shift_range=(-0.1, 0.1),  # posun intenzity
                 intensity_scale_range=(0.9, 1.1),   # škálování intenzity
                 noise_level=0.03):            # úroveň gaussovského šumu
        
        self.rotation_range = rotation_range
        self.flip_probability = flip_probability
        self.intensity_shift_range = intensity_shift_range
        self.intensity_scale_range = intensity_scale_range
        self.noise_level = noise_level
    
    def apply_augmentation(self, input_tensor, target_tensor, mask_tensor):
        """
        Aplikuje stejné augmentace na vstupní tensor, cílový tensor a masku
        
        Args:
            input_tensor: Tensor vstupního obrazu (pseudo-healthy) [C, D, H, W]
            target_tensor: Tensor cílového obrazu (ADC s lézemi) [C, D, H, W]
            mask_tensor: Tensor masky léze [C, D, H, W]
            
        Returns:
            Augmentované tensory (input, target, mask)
        """
        # Ověření, že všechny tensory mají stejný tvar
        assert input_tensor.shape == target_tensor.shape == mask_tensor.shape
        
        device = input_tensor.device
        
        # Převedeme na numpy pro snazší manipulaci
        input_np = input_tensor.cpu().numpy()
        target_np = target_tensor.cpu().numpy()
        mask_np = mask_tensor.cpu().numpy()
        
        # 1. Náhodné rotace
        if self.rotation_range[0] < self.rotation_range[1]:
            # Vybereme náhodný úhel z rozsahu
            angle_x = np.random.uniform(*self.rotation_range)
            angle_y = np.random.uniform(*self.rotation_range)
            angle_z = np.random.uniform(*self.rotation_range)
            
            # Aplikujeme rotaci na všechny kanály
            for c in range(input_np.shape[0]):
                # Rotace kolem osy x
                if abs(angle_x) > 1:
                    input_np[c] = self._rotate_volume(input_np[c], angle_x, axis=0)
                    target_np[c] = self._rotate_volume(target_np[c], angle_x, axis=0)
                    mask_np[c] = self._rotate_volume(mask_np[c], angle_x, axis=0)
                
                # Rotace kolem osy y
                if abs(angle_y) > 1:
                    input_np[c] = self._rotate_volume(input_np[c], angle_y, axis=1)
                    target_np[c] = self._rotate_volume(target_np[c], angle_y, axis=1)
                    mask_np[c] = self._rotate_volume(mask_np[c], angle_y, axis=1)
                
                # Rotace kolem osy z
                if abs(angle_z) > 1:
                    input_np[c] = self._rotate_volume(input_np[c], angle_z, axis=2)
                    target_np[c] = self._rotate_volume(target_np[c], angle_z, axis=2)
                    mask_np[c] = self._rotate_volume(mask_np[c], angle_z, axis=2)
        
        # 2. Náhodné překlápění (flip)
        if np.random.random() < self.flip_probability:
            axis = np.random.choice([0, 1, 2])  # Náhodně vybereme osu
            input_np = np.flip(input_np, axis=axis+1)  # +1 protože první dimenze je kanál
            target_np = np.flip(target_np, axis=axis+1)
            mask_np = np.flip(mask_np, axis=axis+1)
        
        # 3. Úpravy intenzity (pouze pro input a target, ne pro masku)
        # Škálování intenzity
        if self.intensity_scale_range[0] < self.intensity_scale_range[1]:
            scale = np.random.uniform(*self.intensity_scale_range)
            input_np = input_np * scale
            target_np = target_np * scale
            
            # Oříznutí hodnot do platného rozsahu
            input_np = np.clip(input_np, 0, 1)
            target_np = np.clip(target_np, 0, 1)
        
        # Posun intenzity
        if self.intensity_shift_range[0] < self.intensity_shift_range[1]:
            shift = np.random.uniform(*self.intensity_shift_range)
            input_np = input_np + shift
            target_np = target_np + shift
            
            # Oříznutí hodnot do platného rozsahu
            input_np = np.clip(input_np, 0, 1)
            target_np = np.clip(target_np, 0, 1)
        
        # 4. Přidání Gaussovského šumu (pouze pro input, ne pro target nebo masku)
        if self.noise_level > 0:
            noise = np.random.normal(0, self.noise_level, input_np.shape)
            input_np = input_np + noise
            input_np = np.clip(input_np, 0, 1)
        
        # Převod zpět na tensory
        input_tensor = torch.from_numpy(input_np).to(device)
        target_tensor = torch.from_numpy(target_np).to(device)
        # Zajistíme, že maska zůstane binární
        mask_np = (mask_np > 0.5).astype(np.float32)
        mask_tensor = torch.from_numpy(mask_np).to(device)
        
        return input_tensor, target_tensor, mask_tensor
    
    def _rotate_volume(self, volume, angle, axis=0):
        """
        Rotuje 3D objem kolem dané osy
        
        Args:
            volume: 3D numpy array
            angle: Úhel rotace ve stupních
            axis: Osa rotace (0=x, 1=y, 2=z)
            
        Returns:
            Rotovaný objem
        """
        from scipy.ndimage import rotate
        
        # Nerotujeme pokud je úhel příliš malý
        if abs(angle) < 1:
            return volume
        
        # Směr os v scipy.ndimage.rotate je jiný než v numpy array
        axes_map = {
            0: (1, 2),  # rotace kolem x -> rotace v rovině y-z
            1: (0, 2),  # rotace kolem y -> rotace v rovině x-z
            2: (0, 1),  # rotace kolem z -> rotace v rovině x-y
        }
        
        # Aplikujeme rotaci pomocí scipy
        rotated = rotate(volume, angle, axes=axes_map[axis], reshape=False, order=1, mode='nearest')
        
        return rotated

class BrainLesionDataset(Dataset):
    def __init__(self, csv_path, target_shape=(80, 96, 80), patch_overlap=0.5, augmentation=None, training=True):
        """
        Dataset pro trénink patch-based modelu na lézích v mozku.
        
        Args:
            csv_path: Cesta k CSV souboru s daty
            target_shape: Cílový tvar patche (D, H, W)
            patch_overlap: Překrytí mezi sousedními patchi pro zajištění kontinuity lézí (0-1)
        """
        self.data = pd.read_csv(csv_path)
        self.target_shape = target_shape
        self.patch_overlap = patch_overlap
        self.augmentation = augmentation
        self.training = training
        
        # Seznam všech patchů pro všechny subjekty
        self.all_patches = []
        
        # Slovník mapující subjekt_id -> originální data a jejich tvary
        self.subject_data = {}
        
        # Předpočítáme všechny patche pro všechny subjekty
        print("Připravuji patche pro všechny subjekty...")
        for idx, row in self.data.iterrows():
            subject_id = row['subject_id']
            print(f"Zpracovávám subjekt {subject_id} ({idx+1}/{len(self.data)})")
            
            # Načteme originální data bez úprav
            try:
                ph_img = nib.load(row['pseudo_healthy'])
                adc_img = nib.load(row['adc'])
                lesion_img = nib.load(row['lesion'])
                
                # Ověříme, že mají všechny obrazy stejnou afinní matici a rozměry
                if not np.allclose(ph_img.affine, adc_img.affine) or not np.allclose(ph_img.affine, lesion_img.affine):
                    print(f"  VAROVÁNÍ: Obrazy pro subjekt {subject_id} mají rozdílné afinní matice! Provádím resampling...")
                    
                    # Použijeme afinní matici pseudo_healthy jako referenční
                    ref_affine = ph_img.affine
                    ref_shape = ph_img.shape
                    
                    # Resampling adc a lesion obrazů do prostoru referenčního obrazu
                    # Pro ADC použijeme lineární interpolaci
                    adc_data = np.array(adc_img.get_fdata())
                    adc_data = np.clip(adc_data, 0, None)
                    if adc_data.max() > 0: 
                        adc_data = adc_data / adc_data.max()
                    
                    # Pro masku léze použijeme nearest-neighbor interpolaci, aby zůstala binární
                    lesion_data = np.array(lesion_img.get_fdata())
                    lesion_data = (lesion_data > 0.1).astype(np.float32)
                    
                    # Pro převzorkování použijeme nibabel resample_to_img funkci, pokud je dostupná, jinak vlastní implementaci
                    try:
                        from nibabel.processing import resample_to_img
                        
                        # Resampling adc obrazu (lineární interpolace)
                        new_adc_img = resample_to_img(adc_img, ph_img, interpolation='linear')
                        adc_data = np.array(new_adc_img.get_fdata())
                        
                        # Resampling lesion obrazu (nearest neighbor, aby zůstal binární)
                        new_lesion_img = resample_to_img(lesion_img, ph_img, interpolation='nearest')
                        lesion_data = (np.array(new_lesion_img.get_fdata()) > 0.1).astype(np.float32)
                        
                    except (ImportError, AttributeError):
                        print("  nibabel.processing.resample_to_img není dostupný, používám SciPy pro resampling...")
                        
                        # Alternativní převzorkování pomocí scipy
                        from scipy.ndimage import map_coordinates
                        
                        # Vytvoříme mřížku pro transformaci ze zdrojových do cílových souřadnic
                        ijk_dest = np.mgrid[0:ref_shape[0], 0:ref_shape[1], 0:ref_shape[2]]
                        ijk_dest = ijk_dest.reshape(3, -1)
                        
                        # Přidáme homogenní souřadnici
                        ijk_dest_homog = np.vstack((ijk_dest, np.ones((1, ijk_dest.shape[1]))))
                        
                        # Transformace z cílových voxelových souřadnic do světa (mm)
                        xyz_dest = ref_affine @ ijk_dest_homog
                        
                        # Transformace ze světa (mm) do zdrojových voxelových souřadnic
                        ijk_src_adc = np.linalg.inv(adc_img.affine) @ xyz_dest
                        ijk_src_lesion = np.linalg.inv(lesion_img.affine) @ xyz_dest
                        
                        # Převzorkování dat
                        adc_data_resampled = np.zeros(ref_shape)
                        lesion_data_resampled = np.zeros(ref_shape)
                        
                        # Trilineární interpolace pro ADC data
                        map_coordinates(adc_data, ijk_src_adc[:3], output=adc_data_resampled.ravel(), order=1)
                        
                        # Nearest neighbor interpolace pro masku léze
                        map_coordinates(lesion_data, ijk_src_lesion[:3], output=lesion_data_resampled.ravel(), order=0)
                        
                        adc_data = adc_data_resampled
                        lesion_data = (lesion_data_resampled > 0.1).astype(np.float32)
                    
                    # Získáme data z pseudo_healthy
                    ph_data = np.array(ph_img.get_fdata())
                    if ph_data.max() > 0: 
                        ph_data = ph_data / ph_data.max()
                    
                else:
                    # Pokud mají stejné afinní matice, použijeme data přímo
                    ph_data = np.array(ph_img.get_fdata())
                    adc_data = np.array(adc_img.get_fdata())
                    lesion_data = np.array(lesion_img.get_fdata())
                    
                    # Normalizace (jen pro obrazová data)
                    if ph_data.max() > 0: ph_data = ph_data / ph_data.max()
                    if adc_data.max() > 0: adc_data = adc_data / adc_data.max()
                    
                    # Normalizace masek - ujistíme se, že jsou binární (0 nebo 1)
                    lesion_data = (lesion_data > 0.1).astype(np.float32)
                
                if ph_img.shape != adc_img.shape or ph_img.shape != lesion_img.shape:
                    print(f"  VAROVÁNÍ: Obrazy pro subjekt {subject_id} mají rozdílné rozměry! Použiju rozměry z pseudo_healthy.")
                
                # Uložíme originální data a jejich tvar do slovníku
                self.subject_data[subject_id] = {
                    'ph_data': ph_data,
                    'adc_data': adc_data,
                    'lesion_data': lesion_data,
                    'orig_shape': ph_data.shape,
                    'affine': ph_img.affine
                }
                
                # Najdeme všechny léze a jejich centra
                lesion_centers = self._find_all_lesion_centers(lesion_data)
                
                if not lesion_centers:
                    print(f"  Subjekt {subject_id}: Nenalezeny žádné léze, vytvářím patch ve středu objemu")
                    # Pokud nejsou léze, vytvoříme alespoň jeden patch ve středu objemu
                    center = [d // 2 for d in lesion_data.shape]
                    patch_info = {
                        'subject_id': subject_id,
                        'center': center,
                        'has_lesion': False
                    }
                    self.all_patches.append(patch_info)
                else:
                    print(f"  Subjekt {subject_id}: Nalezeno {len(lesion_centers)} center lézí")
                    
                    # Pro každé centrum léze vytvoříme patch
                    for i, center in enumerate(lesion_centers):
                        patch_info = {
                            'subject_id': subject_id,
                            'center': center,
                            'has_lesion': True
                        }
                        self.all_patches.append(patch_info)
                    
                    # Hledáme velké léze, které by mohly vyžadovat více patchů
                    labeled_lesions, num_lesions = ndimage.label(lesion_data)
                    
                    for i in range(1, num_lesions + 1):
                        lesion_mask = labeled_lesions == i
                        lesion_size = np.sum(lesion_mask)
                        
                        # Pokud je léze větší než polovina velikosti patche, může vyžadovat více patchů
                        if lesion_size > np.prod(self.target_shape) * 0.5:
                            print(f"  Subjekt {subject_id}: Léze {i} je velká ({lesion_size} voxelů), generuji dodatečné patche")
                            
                            # ... zbytek kódu pro generování patchů zůstává stejný ...
                            
                            # Najdeme hranice léze
                            indices = np.where(lesion_mask)
                            min_bounds = [np.min(idx) for idx in indices]
                            max_bounds = [np.max(idx) for idx in indices]
                            
                            # Vytvoříme mřížku bodů pro pokrytí léze
                            # Stanovíme krok mezi patchi (s ohledem na překrytí)
                            steps = [int(ts * (1 - self.patch_overlap)) for ts in self.target_shape]
                            
                            # Vypočítáme startovní pozice (rozšířené o polovinu patche na každou stranu)
                            start_positions = []
                            for dim, (min_b, max_b, step, ts) in enumerate(zip(min_bounds, max_bounds, steps, self.target_shape)):
                                half_size = ts // 2
                                start = max(0, min_b - half_size)
                                end = min(lesion_data.shape[dim] - step, max_b + half_size)
                                positions = list(range(start, end, step))
                                if not positions or end > positions[-1]:
                                    positions.append(min(end, lesion_data.shape[dim] - ts))
                                start_positions.append(positions)
                            
                            # Vytvoříme všechny kombinace startovních pozic pro všechny dimenze
                            for start_z, start_y, start_x in product(start_positions[0], start_positions[1], start_positions[2]):
                                # Spočítáme střed patche
                                center = [
                                    start_z + self.target_shape[0] // 2,
                                    start_y + self.target_shape[1] // 2,
                                    start_x + self.target_shape[2] // 2
                                ]
                                
                                # Kontrola, zda patch obsahuje alespoň část léze
                                patch_mask = self._extract_patch(lesion_mask.astype(float), center)
                                if np.sum(patch_mask) > 0:
                                    patch_info = {
                                        'subject_id': subject_id,
                                        'center': center,
                                        'has_lesion': True,
                                        'patch_id': f"lesion{i}_z{start_z}_y{start_y}_x{start_x}"
                                    }
                                    # Přidáme pouze pokud podobný patch ještě neexistuje
                                    if not any(self._is_similar_patch(p, patch_info) for p in self.all_patches):
                                        self.all_patches.append(patch_info)
            
            except Exception as e:
                print(f"  CHYBA při zpracování subjektu {subject_id}: {e}")
        
        print(f"Celkem vytvořeno {len(self.all_patches)} patchů pro {len(self.data)} subjektů")
    
    def _is_similar_patch(self, patch1, patch2, distance_threshold=20):
        """Kontroluje, zda jsou dva patche podobné (blízké centra)"""
        if patch1['subject_id'] != patch2['subject_id']:
            return False
            
        center1 = patch1['center']
        center2 = patch2['center']
        
        # Vypočítáme euklidovskou vzdálenost mezi centry
        distance = np.sqrt(sum((c1 - c2) ** 2 for c1, c2 in zip(center1, center2)))
        return distance < distance_threshold
    
    def __len__(self):
        return len(self.all_patches)
    
    def __getitem__(self, idx):
        """Vrací konkrétní patch podle jeho indexu"""
        patch_info = self.all_patches[idx]
        subject_id = patch_info['subject_id']
        center = patch_info['center']
        
        # Získáme data subjektu ze slovníku
        subject_data = self.subject_data[subject_id]
        
        # Vytvoříme výřez (patch) se středem v lézi
        ph_patch = self._extract_patch(subject_data['ph_data'], center)
        adc_patch = self._extract_patch(subject_data['adc_data'], center)
        lesion_patch = self._extract_patch(subject_data['lesion_data'], center)
        
        # Převod na tensory a přidání dimenze pro kanál
        ph_tensor = torch.from_numpy(ph_patch).float().unsqueeze(0)
        adc_tensor = torch.from_numpy(adc_patch).float().unsqueeze(0)
        lesion_tensor = torch.from_numpy(lesion_patch).float().unsqueeze(0)
        
        if self.augmentation is not None and self.training:
            ph_tensor, adc_tensor, lesion_tensor = self.augmentation.apply_augmentation(
                ph_tensor, adc_tensor, lesion_tensor
            )
        ph_tensor = ph_tensor.float()
        adc_tensor = adc_tensor.float()
        lesion_tensor = lesion_tensor.float()
        
        # Připravíme metadata o patchi pro zpětnou rekonstrukci
        patch_meta = {
            'subject_id': subject_id,
            'center': center,
            'patch_id': patch_info.get('patch_id', f"patch_{idx}"),
            'has_lesion': patch_info['has_lesion'],
            'orig_shape': subject_data['orig_shape'],
            'patch_shape': self.target_shape
        }
        
        return {
            'input': ph_tensor,
            'target': adc_tensor,
            'mask': lesion_tensor,
            'subject_id': subject_id,
            'patch_meta': patch_meta
        }
    
    def _find_all_lesion_centers(self, mask):
        """Najde centra všech souvislých lézí v masce"""
        # Pokud maska neobsahuje léze, vrátíme prázdný seznam
        if np.max(mask) == 0:
            return []
        
        # Označíme souvislé oblasti v binární masce
        binary_mask = mask > 0.1
        labeled_mask, num_features = ndimage.label(binary_mask)
        
        centers = []
        # Pro každou souvislou oblast najdeme centrum
        for i in range(1, num_features + 1):
            region_mask = labeled_mask == i
            if np.sum(region_mask) > 10:  # Ignorujeme velmi malé léze (méně než 10 voxelů)
                center = ndimage.center_of_mass(region_mask)
                centers.append([int(c) for c in center])
        
        return centers
    
    def _extract_patch(self, data, center, padding_value=0):
        """Extrahuje patch pevné velikosti ze zadaných dat kolem zadaného centra bez interpolace"""
        # Převedeme centrum na integer indexy
        center = [int(c) for c in center]
        
        # Vytvoříme prázdný patch cílové velikosti
        patch = np.ones(self.target_shape, dtype=np.float32) * padding_value
        
        # Pro každou dimenzi vypočítáme hranice výřezu
        data_starts = []  # Startovní indexy v původních datech
        data_ends = []    # Koncové indexy v původních datech
        patch_starts = [] # Startovní indexy v patchi
        patch_ends = []   # Koncové indexy v patchi
        
        for dim, (center_pos, target_size, data_size) in enumerate(zip(center, self.target_shape, data.shape)):
            # Polovina velikosti cílového tvaru pro danou dimenzi
            half_size = target_size // 2
            
            # Výpočet hranic v původních datech
            data_start = max(0, center_pos - half_size)
            data_end = min(data_size, center_pos + half_size + (target_size % 2))
            
            # Výpočet odpovídajících hranic v patchi
            patch_start = max(0, half_size - center_pos)
            patch_end = patch_start + (data_end - data_start)
            
            data_starts.append(data_start)
            data_ends.append(data_end)
            patch_starts.append(patch_start)
            patch_ends.append(patch_end)
        
        # Zkopírujeme data z původního obrazu do patche BEZ INTERPOLACE
        patch[patch_starts[0]:patch_ends[0], 
              patch_starts[1]:patch_ends[1], 
              patch_starts[2]:patch_ends[2]] = data[data_starts[0]:data_ends[0], 
                                                   data_starts[1]:data_ends[1], 
                                                   data_starts[2]:data_ends[2]]
        
        return patch
    
    def get_full_subject_data(self, subject_id):
        """Vrátí originální data pro daný subjekt"""
        return self.subject_data.get(subject_id, None)
    
    def get_patch_locations(self, subject_id):
        """Vrátí seznam všech patch lokací pro daný subjekt"""
        return [p for p in self.all_patches if p['subject_id'] == subject_id]
    
    def reconstruct_full_volume(self, patches, patch_centers, orig_shape, blend_weights=True):
        """
        Rekonstruuje plný objem z patchů.
        
        Args:
            patches: Seznam patchů (numpy arrays)
            patch_centers: Seznam center patchů v původním objemu
            orig_shape: Tvar původního objemu
            blend_weights: Zda použít vážené průměrování pro překrývající se oblasti
            
        Returns:
            Rekonstruovaný objem
        """
        # Inicializujeme prázdný výstupní objem a váhový objem pro vážené průměrování
        output_volume = np.zeros(orig_shape, dtype=np.float32)
        weight_volume = np.zeros(orig_shape, dtype=np.float32)
        
        # Pro každý patch
        for patch, center in zip(patches, patch_centers):
            # Vytvoříme váhovou masku pro tento patch
            if blend_weights:
                # Vytvořím Gaussovskou váhu, vyšší ve středu patche a nižší na okrajích
                weight_mask = np.ones(self.target_shape, dtype=np.float32)
                
                # Pro každou dimenzi vytvoříme gaussovský profil
                for dim in range(3):
                    # Vytvoříme souřadnicovou síť pro danou dimenzi
                    coords = np.linspace(-1, 1, self.target_shape[dim])
                    
                    # Gaussovský profil: exp(-x^2/sigma^2)
                    sigma = 0.4  # Šířka gaussovské křivky
                    gauss_profile = np.exp(-(coords**2) / (2 * sigma**2))
                    
                    # Rozšíříme profil do správného tvaru pro násobení
                    if dim == 0:
                        gauss_profile = gauss_profile.reshape(-1, 1, 1)
                    elif dim == 1:
                        gauss_profile = gauss_profile.reshape(1, -1, 1)
                    else:
                        gauss_profile = gauss_profile.reshape(1, 1, -1)
                    
                    # Vynásobíme aktuální váhovou masku gaussovským profilem
                    weight_mask = weight_mask * gauss_profile
                
                # Normalizace vah na rozsah 0-1
                weight_mask = weight_mask / np.max(weight_mask)
            else:
                # Jednotná váha pro celý patch
                weight_mask = np.ones(self.target_shape, dtype=np.float32)
            
            # Vypočítáme hranice pro vložení patche do původního objemu
            data_starts = []
            data_ends = []
            patch_starts = []
            patch_ends = []
            
            for dim, (center_pos, target_size) in enumerate(zip(center, self.target_shape)):
                half_size = target_size // 2
                
                # Výpočet hranic v původním objemu
                data_start = max(0, center_pos - half_size)
                data_end = min(orig_shape[dim], center_pos + half_size + (target_size % 2))
                
                # Výpočet odpovídajících hranic v patchi
                patch_start = max(0, half_size - center_pos)
                patch_end = patch_start + (data_end - data_start)
                
                data_starts.append(data_start)
                data_ends.append(data_end)
                patch_starts.append(patch_start)
                patch_ends.append(patch_end)
            
            # Přidáme vážený patch do výstupního objemu
            output_volume[data_starts[0]:data_ends[0], 
                          data_starts[1]:data_ends[1], 
                          data_starts[2]:data_ends[2]] += \
                patch[patch_starts[0]:patch_ends[0], 
                      patch_starts[1]:patch_ends[1], 
                      patch_starts[2]:patch_ends[2]] * \
                weight_mask[patch_starts[0]:patch_ends[0], 
                           patch_starts[1]:patch_ends[1], 
                           patch_starts[2]:patch_ends[2]]
            
            # Přidáme váhy do váhového objemu
            weight_volume[data_starts[0]:data_ends[0], 
                          data_starts[1]:data_ends[1], 
                          data_starts[2]:data_ends[2]] += \
                weight_mask[patch_starts[0]:patch_ends[0], 
                           patch_starts[1]:patch_ends[1], 
                           patch_starts[2]:patch_ends[2]]
        
        # Normalizujeme výstupní objem váhami (vyhneme se dělení nulou)
        valid_mask = weight_volume > 0
        output_volume[valid_mask] = output_volume[valid_mask] / weight_volume[valid_mask]
        
        return output_volume

def weighted_lesion_loss(outputs, targets, masks, lesion_weight=10.0):
    """
    Váhovaná loss funkce, která klade větší důraz na oblasti lézí.
    
    Args:
        outputs: Výstupy modelu
        targets: Cílové hodnoty
        masks: Binární masky lézí
        lesion_weight: Váha aplikovaná na oblasti lézí (1.0 znamená žádné převážení)
        
    Returns:
        Celková loss
    """
    # Základní L1 loss pro celý obraz
    base_loss = torch.abs(outputs - targets)
    
    # Vytvoříme váhovou mapu: 1.0 pro zdravé oblasti, lesion_weight pro oblasti lézí
    weight_map = 1.0 + (lesion_weight - 1.0) * masks
    
    # Aplikujeme váhovou mapu na loss
    weighted_loss = base_loss * weight_map
    
    # Vrátíme průměr
    return weighted_loss.mean()

# Funkce pro vyhlazení výstupu jako post-processing
def apply_smoothing(outputs, masks, inputs, sigma=0.8, iterations=2):
    """
    Aplikuje gaussovské vyhlazení pouze na okolí léze, ale ne na lézi samotnou.
    Zachovává strukturu léze a vytváří plynulý přechod na hranicích.
    
    Args:
        outputs: Tensor výstupu modelu ve tvaru (B, 1, D, H, W)
        masks: Tensor masek lézí ve tvaru (B, 1, D, H, W)
        inputs: Tensor vstupů ve tvaru (B, 1, D, H, W) - použijeme pro vytvoření masky mozku
        sigma: Parametr rozptylu pro Gaussovské vyhlazení
        iterations: Počet iterací dilatace pro okolí léze
    
    Returns:
        Tensor upraveného výstupu se stejným tvarem jako vstup
    """
    smoothed_results = []
    
    # Převod na numpy pro efektivnější zpracování
    outputs_np = outputs.detach().cpu().numpy()
    masks_np = masks.detach().cpu().numpy()
    inputs_np = inputs.detach().cpu().numpy()
    
    for b in range(outputs.shape[0]):
        # Získání dat pro aktuální batch
        curr_output = outputs_np[b, 0]
        curr_mask = masks_np[b, 0]
        curr_input = inputs_np[b, 0]
        
        # Vytvoření binární masky mozku - všude, kde je nenulová hodnota vstupu
        brain_mask = (curr_input > 0.01).astype(np.float32)
        
        # Vytvoření binární masky léze
        lesion_mask = (curr_mask > 0.1).astype(np.float32)
        
        # Vytvoření dilatované masky pro okolí léze
        dilated_mask = binary_dilation(lesion_mask, iterations=iterations).astype(np.float32)
        
        # Vytvoření masky pouze pro okolí léze (dilatovaná oblast minus původní léze)
        # Toto je klíčová změna - oddělíme okolí léze od samotné léze
        transition_zone_mask = dilated_mask - lesion_mask
        
        # Zajistíme, že přechodová zóna zůstane uvnitř masky mozku
        transition_zone_mask = transition_zone_mask * brain_mask
        
        # Aplikace gaussovského vyhlazení na celý výstup
        smoothed = gaussian_filter(curr_output, sigma=sigma)
        
        # Zajistíme, že vyhlazený výstup zůstane uvnitř masky mozku
        smoothed = smoothed * brain_mask
        
        # Kombinace:
        # 1. Původní výstup v místech mimo přechodovou zónu (včetně lézí)
        # 2. Vyhlazený výstup v přechodové zóně
        final_output = curr_output * (1 - transition_zone_mask) + smoothed * transition_zone_mask
        
        # Převod zpět na tensor
        smoothed_results.append(torch.from_numpy(final_output).to(outputs.device))
    
    # Složení výsledného batch tensoru
    result = torch.stack(smoothed_results).unsqueeze(1)
    
    return result

def calculate_lesion_metrics(outputs, targets, masks, inputs=None):
    """
    Vypočítá metriky zaměřené specificky na kvalitu rekonstrukce lézí.
    
    Args:
        outputs: Výstupy modelu
        targets: Cílové ADC mapy
        masks: Binární masky lézí
        inputs: Původní pseudozdravý obraz (volitelný)
        
    Returns:
        Dictionary s metrikami pro léze
    """
    # Převod na numpy pro výpočet
    outputs_np = outputs.detach().cpu().numpy()
    targets_np = targets.detach().cpu().numpy()
    masks_np = masks.detach().cpu().numpy() > 0.5
    
    # Pokud nemáme žádné léze, vrátíme prázdný slovník
    if np.sum(masks_np) == 0:
        if inputs is None:
            return {'lesion_mae': 0.0}
        else:
            return {'lesion_mae': 0.0, 'input_output_lesion_mae': 0.0}
    
    # MAE pouze pro oblasti lézí (výstup vs. cíl)
    lesion_mae = np.sum(np.abs(outputs_np[masks_np] - targets_np[masks_np])) / np.sum(masks_np)
    
    # Pokud máme k dispozici původní vstupy, počítáme také MAE mezi vstupem a výstupem v oblasti léze
    if inputs is not None:
        inputs_np = inputs.detach().cpu().numpy()
        input_output_lesion_mae = np.sum(np.abs(outputs_np[masks_np] - inputs_np[masks_np])) / np.sum(masks_np)
        return {'lesion_mae': float(lesion_mae), 'input_output_lesion_mae': float(input_output_lesion_mae)}
    
    return {'lesion_mae': float(lesion_mae)}

class HIELesionInpainter(nn.Module):
    def __init__(self, checkpoint_path, device="cpu"):
        super(HIELesionInpainter, self).__init__()
        
        # Vytvoření modelu s přesně stejnou architekturou jako brain2vec
        self.vae = AutoencoderKL(
            spatial_dims=3, 
            in_channels=1, 
            out_channels=1, 
            latent_channels=1,
            num_channels=(64, 128, 128, 128),
            num_res_blocks=2, 
            norm_num_groups=32,
            norm_eps=1e-06,
            attention_levels=(False, False, False, False), 
            with_decoder_nonlocal_attn=False, 
            with_encoder_nonlocal_attn=False
        )
        
        print(f"Načítání checkpoint z: {checkpoint_path}")
        try:
            # Načtení vah z checkpointu brain2vec
            state_dict = torch.load(checkpoint_path, map_location=device)
            
            # Zobrazíme informace o načteném state_dict
            sample_keys = list(state_dict.keys())[:5]
            
            # Pokus o načtení vah
            missing, unexpected = self.vae.load_state_dict(state_dict, strict=False)
            print(f"Načítání vah - chybějící klíče: {len(missing)}, neočekávané klíče: {len(unexpected)}")
            if len(missing) > 0:
                print(f"Příklady chybějících klíčů: {missing[:3]}")
            if len(unexpected) > 0:
                print(f"Příklady neočekávaných klíčů: {unexpected[:3]}")
            
            print("Váhy částečně načteny s tolerancí k rozdílům v architektuře")
            
        except Exception as e:
            print(f"Chyba při načítání vah: {e}")
            import traceback
            traceback.print_exc()
            print("Pokračuji s náhodně inicializovanými vahami")
        
        # Zmrazíme encoder pro fine-tuning
        for name, param in self.vae.named_parameters():
            if "encoder" in name:
                # Odmrazíme pouze poslední vrstvy enkodéru
                # Hledáme vrstvy ve vzorech jako "encoder.down.2" nebo "encoder.down.3" nebo "encoder.mid"
                if any(pattern in name for pattern in ["encoder.down.2", "encoder.down.3", "encoder.mid"]):
                    param.requires_grad = True
                    print(f"Odmrazena vrstva: {name}")
                else:
                    param.requires_grad = False
                    
        print("Encoder vrstvy částečně odmrazeny - poslední vrstvy jsou aktivní pro trénink")
    
    def forward(self, x, mask):
        # Pokud jsme v trénovacím módu a používáme perceptuální ztrátu,
        # potřebujeme gradienty z enkodéru
        if self.training:
            z_mu, z_sigma = self.vae.encode(x)
        else:
            with torch.no_grad():
                z_mu, z_sigma = self.vae.encode(x)
        
        # Zbytek kódu zůstává stejný
        decoded = self.vae.decode(z_mu)
        if decoded.shape != x.shape:
            decoded = F.interpolate(decoded, size=x.shape[2:], mode='trilinear', align_corners=False)
        raw_output = x * (1 - mask) + decoded * mask
        return raw_output

# Funkce pro vyhlazení výstupu jako post-processing
def apply_smoothing(outputs, masks, inputs, sigma=0.8, iterations=2):
    """
    Aplikuje gaussovské vyhlazení pouze na okolí léze, ale ne na lézi samotnou.
    Zachovává strukturu léze a vytváří plynulý přechod na hranicích.
    
    Args:
        outputs: Tensor výstupu modelu ve tvaru (B, 1, D, H, W)
        masks: Tensor masek lézí ve tvaru (B, 1, D, H, W)
        inputs: Tensor vstupů ve tvaru (B, 1, D, H, W) - použijeme pro vytvoření masky mozku
        sigma: Parametr rozptylu pro Gaussovské vyhlazení
        iterations: Počet iterací dilatace pro okolí léze
    
    Returns:
        Tensor upraveného výstupu se stejným tvarem jako vstup
    """
    smoothed_results = []
    
    # Převod na numpy pro efektivnější zpracování
    outputs_np = outputs.detach().cpu().numpy()
    masks_np = masks.detach().cpu().numpy()
    inputs_np = inputs.detach().cpu().numpy()
    
    for b in range(outputs.shape[0]):
        # Získání dat pro aktuální batch
        curr_output = outputs_np[b, 0]
        curr_mask = masks_np[b, 0]
        curr_input = inputs_np[b, 0]
        
        # Vytvoření binární masky mozku - všude, kde je nenulová hodnota vstupu
        brain_mask = (curr_input > 0.01).astype(np.float32)
        
        # Vytvoření binární masky léze
        lesion_mask = (curr_mask > 0.1).astype(np.float32)
        
        # Vytvoření dilatované masky pro okolí léze
        dilated_mask = binary_dilation(lesion_mask, iterations=iterations).astype(np.float32)
        
        # Vytvoření masky pouze pro okolí léze (dilatovaná oblast minus původní léze)
        # Toto je klíčová změna - oddělíme okolí léze od samotné léze
        transition_zone_mask = dilated_mask - lesion_mask
        
        # Zajistíme, že přechodová zóna zůstane uvnitř masky mozku
        transition_zone_mask = transition_zone_mask * brain_mask
        
        # Aplikace gaussovského vyhlazení na celý výstup
        smoothed = gaussian_filter(curr_output, sigma=sigma)
        
        # Zajistíme, že vyhlazený výstup zůstane uvnitř masky mozku
        smoothed = smoothed * brain_mask
        
        # Kombinace:
        # 1. Původní výstup v místech mimo přechodovou zónu (včetně lézí)
        # 2. Vyhlazený výstup v přechodové zóně
        final_output = curr_output * (1 - transition_zone_mask) + smoothed * transition_zone_mask
        
        # Převod zpět na tensor
        smoothed_results.append(torch.from_numpy(final_output).to(outputs.device))
    
    # Složení výsledného batch tensoru
    result = torch.stack(smoothed_results).unsqueeze(1)
    
    return result

def visualize_full_subject(dataset, model, subject_id, epoch, output_dir, device):
    """
    Vytvoří a uloží vizualizace pro celý objem mozku jednoho subjektu.
    
    Args:
        dataset: Instance BrainLesionDataset
        model: Natrénovaný model HIELesionInpainter
        subject_id: ID subjektu pro vizualizaci
        epoch: Číslo epochy
        output_dir: Adresář pro uložení vizualizací
        device: Zařízení pro běh modelu (CPU/GPU)
    """
    from matplotlib.backends.backend_pdf import PdfPages
    
    viz_dir = os.path.join(output_dir, 'visualizations')
    os.makedirs(viz_dir, exist_ok=True)
    
    print(f"Vizualizuji celý subjekt {subject_id} pro epochu {epoch}...")
    
    # Získám originální data subjektu
    subject_data = dataset.get_full_subject_data(subject_id)
    if subject_data is None:
        print(f"  VAROVÁNÍ: Data pro subjekt {subject_id} nenalezena!")
        return
    
    ph_data = subject_data['ph_data']
    adc_data = subject_data['adc_data']
    lesion_data = subject_data['lesion_data']
    orig_shape = subject_data['orig_shape']
    
    # Získám všechny patche pro tento subjekt
    patch_locations = dataset.get_patch_locations(subject_id)
    
    # Připravím patche pro rekonstrukci
    output_patches = []
    patch_centers = []
    
    # Pro každou lokaci patche
    for patch_info in patch_locations:
        center = patch_info['center']
        
        # Extrakce patche pro vstup do modelu
        ph_patch = dataset._extract_patch(ph_data, center)
        lesion_patch = dataset._extract_patch(lesion_data, center)
        
        # Převod na tensory
        ph_tensor = torch.from_numpy(ph_patch).float().unsqueeze(0).unsqueeze(0).to(device)
        lesion_tensor = torch.from_numpy(lesion_patch).float().unsqueeze(0).unsqueeze(0).to(device)
        
        # Inference modelu
        with torch.no_grad():
            raw_output = model(ph_tensor, lesion_tensor)
            smooth_output = apply_smoothing(raw_output, lesion_tensor, ph_tensor)
        
        # Uložím výstupní patch a jeho centrum
        output_patches.append(smooth_output[0, 0].cpu().numpy())
        patch_centers.append(center)
    
    # Rekonstrukce plného objemu
    reconstructed_output = dataset.reconstruct_full_volume(
        output_patches, patch_centers, orig_shape, blend_weights=True
    )
    
    # Najdeme všechny řezy, které obsahují léze
    z_slices_with_lesions = np.where(np.any(lesion_data > 0.1, axis=(1, 2)))[0]
    
    # Pokud nemáme žádné léze, zobrazíme alespoň jeden řez ve středu objemu
    if len(z_slices_with_lesions) == 0:
        z_slices_with_lesions = [orig_shape[0] // 2]
        print(f"Subjekt {subject_id}: Nenalezeny žádné léze")
    else:
        print(f"Subjekt {subject_id}: Nalezeno {len(z_slices_with_lesions)} řezů s lézemi")
    
    # Výpočet MAE pro celý objem s lézemi
    mask_volume = lesion_data > 0.1
    lesion_voxel_count = np.sum(mask_volume)
    
    # SOUBOR - PDF s detailními vizualizacemi všech řezů
    pdf_filename = f'epoch_{epoch:03d}_subject_{subject_id}_all_slices.pdf'
    pdf_path = os.path.join(viz_dir, pdf_filename)
    
    with PdfPages(pdf_path) as pdf:
        # Nejprve vytvoříme souhrnnou stránku s informacemi
        plt.figure(figsize=(12, 8))
        plt.text(0.5, 0.5, 
                 f"Vizualizace pro epochu {epoch}\nSubjekt: {subject_id}\n"
                 f"Počet řezů s lézemi: {len(z_slices_with_lesions)}\n"
                 f"Celkový počet voxelů s lézí: {lesion_voxel_count}\n", 
                 ha='center', va='center', fontsize=16)
        plt.axis('off')
        pdf.savefig()
        plt.close()
        
        # Pro každý řez s lézí vytvoříme vizualizaci
        slice_mae_values = []
        
        for z_slice in z_slices_with_lesions:
            # Vytvoříme figure se 4 subplot podle požadavků
            fig, axes = plt.subplots(2, 2, figsize=(16, 12))
            
            # Normalizace pro lepší vizualizaci
            norm = Normalize(vmin=0, vmax=1)
            
            # 1. Output modelu (vpravo nahoře)
            im1 = axes[0, 1].imshow(reconstructed_output[z_slice], cmap='gray', norm=norm)
            axes[0, 1].set_title('1. Výstup modelu (rekonstrukce)')
            axes[0, 1].axis('off')
            plt.colorbar(im1, ax=axes[0, 1], fraction=0.046, pad=0.04)
            
            # 2. Target mapa (ADC s lézemi) (vpravo dole)
            im2 = axes[1, 1].imshow(adc_data[z_slice], cmap='gray', norm=norm)
            axes[1, 1].set_title('2. Target (ADC s lézemi)')
            axes[1, 1].axis('off')
            plt.colorbar(im2, ax=axes[1, 1], fraction=0.046, pad=0.04)
            
            # 3. Původní pseudozdravý obraz (vlevo nahoře)
            im3 = axes[0, 0].imshow(ph_data[z_slice], cmap='gray', norm=norm)
            axes[0, 0].set_title('3. Původní pseudozdravý obraz')
            axes[0, 0].axis('off')
            plt.colorbar(im3, ax=axes[0, 0], fraction=0.046, pad=0.04)
            
            # 4. Pseudozdravý obraz se zakreslenými lézemi (vlevo dole)
            # Vytvoříme RGB obrázek s červeně vyznačenými lézemi
            overlay = np.zeros((orig_shape[1], orig_shape[2], 3))
            overlay[:, :, 0] = ph_data[z_slice]  # R kanál - vstup
            overlay[:, :, 1] = ph_data[z_slice]  # G kanál - vstup
            overlay[:, :, 2] = ph_data[z_slice]  # B kanál - vstup
            
            # Přidáme léze jako červené oblasti
            mask_slice = lesion_data[z_slice]
            overlay[mask_slice > 0.1, 0] = 1.0  # Červená pro léze
            overlay[mask_slice > 0.1, 1] = 0.0  # Snížíme zelenou
            overlay[mask_slice > 0.1, 2] = 0.0  # Snížíme modrou
            
            im4 = axes[1, 0].imshow(overlay)
            axes[1, 0].set_title('4. Pseudozdravý obraz s vyznačenými lézemi')
            axes[1, 0].axis('off')
            
            # Výpočet MAE pro aktuální řez, pouze v oblasti léze
            mask_region_slice = mask_slice > 0.1
            lesion_pixels_in_slice = np.sum(mask_region_slice)
            
            if lesion_pixels_in_slice > 0:
                mae_lesion_slice = np.mean(np.abs(reconstructed_output[z_slice][mask_region_slice] - adc_data[z_slice][mask_region_slice]))
                input_vs_output_mae = np.mean(np.abs(reconstructed_output[z_slice][mask_region_slice] - ph_data[z_slice][mask_region_slice]))
                slice_mae_values.append((z_slice, mae_lesion_slice, input_vs_output_mae))
                
                # Výpočet MAE pro celý objem s lézemi, jen pokud máme léze
                if lesion_voxel_count > 0:
                    mae_lesion_volume = np.sum(np.abs(reconstructed_output[mask_volume] - adc_data[mask_volume])) / lesion_voxel_count
                    input_vs_output_mae = np.mean(np.abs(reconstructed_output[z_slice][mask_region_slice] - ph_data[z_slice][mask_region_slice]))
                    
                    # Přidáme text s MAE skóre pro léze
                    fig.suptitle(f'Řez: {z_slice}\n'
                            f'MAE (rekonstrukce vs. target, léze): {mae_lesion_slice:.4f}\n'
                            f'MAE (rekonstrukce vs. pseudozdravý, léze): {input_vs_output_mae:.4f}\n'
                            f'Počet pixelů s lézí v tomto řezu: {lesion_pixels_in_slice} z celkem {lesion_voxel_count} voxelů', 
                            fontsize=16)
                else:
                    fig.suptitle(f'Řez: {z_slice}\n'
                                f'V tomto řezu jsou léze, ale v celém objemu není dostatek lézí pro MAE\n'
                                f'Počet pixelů s lézí v tomto řezu: {lesion_pixels_in_slice}', 
                                fontsize=16)
            else:
                fig.suptitle(f'Řez: {z_slice}\n'
                            f'V tomto řezu nejsou léze', 
                            fontsize=16)
            
            plt.tight_layout(rect=[0, 0, 1, 0.96])  # Zajistíme místo pro nadpis
            
            # Přidáme obrázek do PDF
            pdf.savefig(fig)
            plt.close(fig)
        
        # Vytvoříme také souhrnnou vizualizaci vývoje MAE pro tento subjekt
        if len(slice_mae_values) > 0:
            plt.figure(figsize=(12, 8))
            
            # Rozbalíme hodnoty z seznamu
            z_values, output_vs_target_values, input_vs_output_values = zip(*slice_mae_values)
            
            # Graf s dvěma y-osami pro lepší zobrazení
            fig, ax1 = plt.subplots(figsize=(14, 8))
            
            # První osa - MAE output vs. target (modrá)
            color = 'tab:blue'
            ax1.set_xlabel('Číslo řezu')
            ax1.set_ylabel('MAE (rekonstrukce vs. target)', color=color)
            ax1.plot(z_values, output_vs_target_values, 'o-', color=color, markersize=8, label='MAE (rekonstrukce vs. target)')
            ax1.tick_params(axis='y', labelcolor=color)
            
            # Druhá osa - MAE input vs. output (červená)
            ax2 = ax1.twinx()
            color = 'tab:red'
            ax2.set_ylabel('MAE (pseudozdravý vs. rekonstrukce)', color=color)
            ax2.plot(z_values, input_vs_output_values, 'o-', color=color, markersize=8, label='MAE (pseudozdravý vs. rekonstrukce)')
            ax2.tick_params(axis='y', labelcolor=color)
            
            # Přidáme průměrné hodnoty jako horizontální čáry
            avg_output_vs_target = np.mean(output_vs_target_values)
            avg_input_vs_output = np.mean(input_vs_output_values)
            
            ax1.axhline(y=avg_output_vs_target, color='tab:blue', linestyle='--', 
                    label=f'Průměr rekonstrukce vs. target: {avg_output_vs_target:.4f}')
            ax2.axhline(y=avg_input_vs_output, color='tab:red', linestyle='--',
                    label=f'Průměr pseudozdravý vs. rekonstrukce: {avg_input_vs_output:.4f}')
            
            # Sloučíme legendy z obou os
            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2)
            
            plt.title(f'Porovnání metrik MAE v jednotlivých řezech s lézemi')
            plt.grid(True, alpha=0.3)
            fig.tight_layout()
            
            # Přidáme graf do PDF
            pdf.savefig(fig)
            plt.close(fig)
    
    print(f"Vytvořen PDF soubor: {pdf_path}")

def visualize_results(inputs, targets, masks, outputs, epoch, output_dir, subject_ids=None):
    """
    Vytvoří a uloží 3 typy vizualizací pro každou epochu:
    1. PDF soubor s detailními vizualizacemi všech řezů
    2. PNG soubor s přehledným grafem MAE pro všechny subjekty
    3. CSV soubor s numerickými výsledky pro další analýzu
    
    Args:
        inputs: Tensor vstupů (pseudo-zdravých obrazů)
        targets: Tensor cílů (ADC s lézemi)
        masks: Tensor masek lézí
        outputs: Tensor výstupů modelu
        epoch: Číslo epochy
        output_dir: Adresář pro uložení vizualizací
        subject_ids: Volitelně ID subjektů pro pojmenování souborů
    """
    from matplotlib.backends.backend_pdf import PdfPages
    import pandas as pd
    
    viz_dir = os.path.join(output_dir, 'visualizations')
    os.makedirs(viz_dir, exist_ok=True)
    print(f"Vytvářím vizualizace pro epochu {epoch} do adresáře {viz_dir}")
    
    # Převod na numpy pro vizualizaci
    inputs_np = inputs.detach().cpu().numpy()
    targets_np = targets.detach().cpu().numpy()
    masks_np = masks.detach().cpu().numpy()
    outputs_np = outputs.detach().cpu().numpy()
    
    batch_size = inputs.shape[0]
    
    # Data pro CSV a souhrnný graf
    summary_data = []
    
    # Pro každý subjekt v dávce
    for b in range(batch_size):
        # Najdeme všechny řezy, které obsahují léze
        z_slices_with_lesions = np.where(np.any(masks_np[b, 0] > 0.1, axis=(1, 2)))[0]
        
        # Pokud nemáme žádné léze, zobrazíme alespoň jeden řez ve středu objemu
        if len(z_slices_with_lesions) == 0:
            z_slices_with_lesions = [inputs_np.shape[2] // 2]
            print(f"Subjekt {subject_ids[b] if subject_ids is not None else f'batch_{b}'}: Nenalezeny žádné léze")
        else:
            print(f"Subjekt {subject_ids[b] if subject_ids is not None else f'batch_{b}'}: Nalezeno {len(z_slices_with_lesions)} řezů s lézemi")
        
        # Výpočet MAE pro celý objem s lézemi
        mask_volume = masks_np[b, 0] > 0.1
        lesion_voxel_count = np.sum(mask_volume)
        
        if lesion_voxel_count > 0:
            mae_lesion_volume = np.sum(np.abs(outputs_np[b, 0][mask_volume] - targets_np[b, 0][mask_volume])) / lesion_voxel_count
            
            # Uložíme data pro CSV
            subject_info = {
                'epoch': epoch,
                'subject_id': subject_ids[b] if subject_ids is not None else f'batch_{b}',
                'lesion_voxel_count': lesion_voxel_count,
                'mae_lesion_volume': mae_lesion_volume,
                'slices_with_lesions': len(z_slices_with_lesions)
            }
            
            # Přidáme MAE pro každý řez s lézemi
            slice_mae_values = []
            for z in z_slices_with_lesions:
                mask_region = masks_np[b, 0, z] > 0.1
                if np.any(mask_region):
                    slice_mae = np.mean(np.abs(outputs_np[b, 0, z][mask_region] - targets_np[b, 0, z][mask_region]))
                    slice_mae_values.append((z_slice, mae_lesion_slice, input_vs_output_mae))
                    subject_info[f'slice_{z}_mae'] = slice_mae
                    subject_info[f'slice_{z}_lesion_pixels'] = np.sum(mask_region)
            
            summary_data.append(subject_info)
        else:
            mae_lesion_volume = 0
        
        # 1. SOUBOR - PDF s detailními vizualizacemi všech řezů
        if subject_ids is not None:
            pdf_filename = f'epoch_{epoch:03d}_subject_{subject_ids[b]}_all_slices.pdf'
        else:
            pdf_filename = f'epoch_{epoch:03d}_batch_{b:02d}_all_slices.pdf'
        
        pdf_path = os.path.join(viz_dir, pdf_filename)
        
        with PdfPages(pdf_path) as pdf:
            # Nejprve vytvoříme souhrnnou stránku s informacemi
            plt.figure(figsize=(12, 8))
            plt.text(0.5, 0.5, 
                     f"Vizualizace pro epochu {epoch}\nSubjekt: {subject_ids[b] if subject_ids is not None else f'batch_{b}'}\n"
                     f"Počet řezů s lézemi: {len(z_slices_with_lesions)}\n"
                     f"Celkový počet voxelů s lézí: {lesion_voxel_count}\n"
                     f"MAE pro celý objem lézí: {mae_lesion_volume:.4f}", 
                     ha='center', va='center', fontsize=16)
            plt.axis('off')
            pdf.savefig()
            plt.close()
            
            # Pro každý řez s lézí vytvoříme vizualizaci
            for z_slice in z_slices_with_lesions:
                # Vytvoříme figure se 4 subplot podle požadavků
                fig, axes = plt.subplots(2, 2, figsize=(16, 12))
                
                # Normalizace pro lepší vizualizaci
                norm = Normalize(vmin=0, vmax=1)
                
                # 1. Output modelu (vpravo nahoře)
                im1 = axes[0, 1].imshow(outputs_np[b, 0, z_slice], cmap='gray', norm=norm)
                axes[0, 1].set_title('1. Výstup modelu (rekonstrukce)')
                axes[0, 1].axis('off')
                plt.colorbar(im1, ax=axes[0, 1], fraction=0.046, pad=0.04)
                
                # 2. Target mapa (ADC s lézemi) (vpravo dole)
                im2 = axes[1, 1].imshow(targets_np[b, 0, z_slice], cmap='gray', norm=norm)
                axes[1, 1].set_title('2. Target (ADC s lézemi)')
                axes[1, 1].axis('off')
                plt.colorbar(im2, ax=axes[1, 1], fraction=0.046, pad=0.04)
                
                # 3. Původní pseudozdravý obraz (vlevo nahoře)
                im3 = axes[0, 0].imshow(inputs_np[b, 0, z_slice], cmap='gray', norm=norm)
                axes[0, 0].set_title('3. Původní pseudozdravý obraz')
                axes[0, 0].axis('off')
                plt.colorbar(im3, ax=axes[0, 0], fraction=0.046, pad=0.04)
                
                # 4. Pseudozdravý obraz se zakreslenými lézemi (vlevo dole)
                # Vytvoříme RGB obrázek s červeně vyznačenými lézemi
                overlay = np.zeros((inputs_np.shape[3], inputs_np.shape[4], 3))
                overlay[:, :, 0] = inputs_np[b, 0, z_slice]  # R kanál - vstup
                overlay[:, :, 1] = inputs_np[b, 0, z_slice]  # G kanál - vstup
                overlay[:, :, 2] = inputs_np[b, 0, z_slice]  # B kanál - vstup
                
                # Přidáme léze jako červené oblasti
                mask_slice = masks_np[b, 0, z_slice]
                overlay[mask_slice > 0.1, 0] = 1.0  # Červená pro léze
                overlay[mask_slice > 0.1, 1] = 0.0  # Snížíme zelenou
                overlay[mask_slice > 0.1, 2] = 0.0  # Snížíme modrou
                
                im4 = axes[1, 0].imshow(overlay)
                axes[1, 0].set_title('4. Pseudozdravý obraz s vyznačenými lézemi')
                axes[1, 0].axis('off')
                
                # Výpočet MAE pro aktuální řez, pouze v oblasti léze
                mask_region_slice = mask_slice > 0.1
                lesion_pixels_in_slice = np.sum(mask_region_slice)
                
                if lesion_pixels_in_slice > 0:
                    # MAE mezi výstupem modelu a cílem (ADC s lézemi)
                    mae_lesion_slice = np.mean(np.abs(reconstructed_output[z_slice][mask_region_slice] - adc_data[z_slice][mask_region_slice]))
                    
                    # MAE mezi výstupem modelu a původním pseudozdravým obrazem
                    input_vs_output_mae = np.mean(np.abs(reconstructed_output[z_slice][mask_region_slice] - ph_data[z_slice][mask_region_slice]))
                    
                    slice_mae_values.append((z_slice, mae_lesion_slice, input_vs_output_mae))
                    
                    # Přidáme text s MAE skóre pro léze
                    fig.suptitle(f'Řez: {z_slice}\n'
                                f'MAE (rekonstrukce vs. target, léze): {mae_lesion_slice:.4f}\n'
                                f'MAE (rekonstrukce vs. pseudozdravý, léze): {input_vs_output_mae:.4f}\n'
                                f'Počet pixelů s lézí v tomto řezu: {lesion_pixels_in_slice} z celkem {lesion_voxel_count} voxelů', 
                                fontsize=16)
                    
                    # Přidáme text s MAE skóre pro léze
                    fig.suptitle(f'Řez: {z_slice}\n'
                                f'MAE (pouze léze - celý objem): {mae_lesion_volume:.4f}, MAE (pouze léze - tento řez): {mae_lesion_slice:.4f}\n'
                                f'Počet pixelů s lézí v tomto řezu: {lesion_pixels_in_slice} z celkem {lesion_voxel_count} voxelů', 
                                fontsize=16)
                else:
                    fig.suptitle(f'Řez: {z_slice}\n'
                                f'MAE (pouze léze - celý objem): {mae_lesion_volume:.4f}\n'
                                f'V tomto řezu nejsou léze, celkem {lesion_voxel_count} voxelů s lézí', 
                                fontsize=16)
                
                plt.tight_layout(rect=[0, 0, 1, 0.96])  # Zajistíme místo pro nadpis
                
                # Přidáme obrázek do PDF
                pdf.savefig(fig)
                plt.close(fig)
            
            # Vytvoříme také souhrnnou vizualizaci vývoje MAE pro tento subjekt
            if len(z_slices_with_lesions) > 0 and lesion_voxel_count > 0:
                if slice_mae_values:
                    plt.figure(figsize=(12, 6))
                    z_values, mae_values = zip(*slice_mae_values)
                    plt.plot(z_values, mae_values, 'o-', markersize=8)
                    plt.axhline(y=mae_lesion_volume, color='r', linestyle='--', label=f'Průměrné MAE celého objemu: {mae_lesion_volume:.4f}')
                    plt.xlabel('Číslo řezu')
                    plt.ylabel('MAE (pouze oblast léze)')
                    plt.title(f'MAE v jednotlivých řezech s lézemi')
                    plt.grid(True)
                    plt.legend()
                    
                    # Přidáme graf do PDF
                    pdf.savefig()
                    plt.close()
        
        print(f"Vytvořen PDF soubor: {pdf_path}")
    
    # 2. SOUBOR - PNG se souhrnným grafem MAE pro všechny subjekty
    if summary_data:
        plt.figure(figsize=(15, 10))
        
        # Vytvoříme skupinový sloupcový graf pro MAE všech subjektů
        subject_ids_plot = [data['subject_id'] for data in summary_data]
        mae_values_plot = [data['mae_lesion_volume'] for data in summary_data]
        
        plt.bar(subject_ids_plot, mae_values_plot, color='skyblue')
        plt.axhline(y=np.mean(mae_values_plot), color='r', linestyle='--', 
                   label=f'Průměrné MAE všech subjektů: {np.mean(mae_values_plot):.4f}')
        
        plt.xlabel('Subjekt')
        plt.ylabel('MAE (pouze oblast léze)')
        plt.title(f'Epocha {epoch}: MAE pro všechny subjekty')
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.legend()
        plt.tight_layout()
        
        # Uložíme souhrnný graf jako PNG
        summary_plot_path = os.path.join(viz_dir, f'epoch_{epoch:03d}_summary_mae.png')
        plt.savefig(summary_plot_path, dpi=150)
        plt.close()
        print(f"Vytvořen souhrnný graf: {summary_plot_path}")
    
    # 3. SOUBOR - CSV s číselnými výsledky
    if summary_data:
        # Uložíme data do CSV
        summary_df = pd.DataFrame(summary_data)
        csv_path = os.path.join(viz_dir, f'epoch_{epoch:03d}_metrics.csv')
        summary_df.to_csv(csv_path, index=False)
        print(f"Vytvořen CSV soubor s metrikami: {csv_path}")
    
    print(f"Vizualizace pro epochu {epoch} dokončeny.")

from perceptual_loss import PerceptualLoss3D  # Přidat tento import na začátek souboru

def train(args):
    # Vytvoření adresářů a TensorBoard writer
    os.makedirs(args.output_dir, exist_ok=True)
    writer = SummaryWriter(os.path.join(args.output_dir, 'logs'))

    # Inicializace augmentace
    augmentation = BrainLesionAugmentation(
        rotation_range=(-15, 15),        # Rotace až ±15 stupňů
        flip_probability=0.5,            # 50% šance na flip
        intensity_shift_range=(-0.1, 0.1), # Posun intenzity ±0.1
        intensity_scale_range=(0.9, 1.1),  # Škálování intenzity ±10%
        noise_level=0.03                 # 3% šumu
    )
    
    # Vytvoření datasetu a dataloaderu
    dataset = BrainLesionDataset(args.csv_path, augmentation=augmentation)
    if len(dataset) == 0:
        print("VAROVÁNÍ: Dataset je prázdný! Zkontrolujte CSV soubor.")
        return
    
    train_size = max(1, int(0.8 * len(dataset)))
    val_size = max(1, len(dataset) - train_size)
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)
    
    # Inicializace zařízení a modelu
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Používám zařízení: {device}")
    model = HIELesionInpainter(args.checkpoint_path, device=device).to(device)
    
    # Inicializace loss funkcí
    mse_loss = nn.MSELoss()
    l1_loss = nn.L1Loss()
    
    # Inicializace perceptual loss
    perceptual_loss = PerceptualLoss3D(num_slices=3)
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        perceptual_loss = perceptual_loss.cuda()
    # Koeficient pro vážení perceptual loss (hodnotu lze experimentálně upravit)
    perceptual_weight = 0.1
    
    # Inicializace optimizeru – filtrujeme pouze parametry, které vyžadují gradient
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=args.lr)
    
    # Hlavní tréninkový cyklus (změna: použití args.num_epochs dle instrukcí)
    for epoch in range(args.num_epochs):
        model.train()
        total_recon_loss = 0
        total_perceptual_loss = 0
        
        for i, batch in enumerate(train_loader):
            inputs = batch['input'].to(device)
            targets = batch['target'].to(device)
            masks = batch['mask'].to(device)
            
            # Forward pass – předpokládáme, že model vrací přímo rekonstruované výstupy
            outputs = model(inputs, masks)
            
            # Výpočet rekonstrukční loss pomocí MSE
            recon_loss = mse_loss(outputs, targets)
            
            # Výpočet perceptual loss
            p_loss = perceptual_loss(outputs, targets)
            
            # Celková loss jako kombinace obou složek
            loss = recon_loss + perceptual_weight * p_loss
            
            # Pro výpis do logu akumulujeme hodnoty loss
            total_recon_loss += recon_loss.item()
            total_perceptual_loss += p_loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if i % 5 == 0:
                print(f"Epoch {epoch+1}/{args.num_epochs}, Batch {i}/{len(train_loader)}, Loss: {loss.item():.4f}")
        
        avg_recon_loss = total_recon_loss / len(train_loader)
        avg_perceptual_loss = total_perceptual_loss / len(train_loader)
        
        print(f"Epoch {epoch+1}/{args.num_epochs}, Recon Loss: {avg_recon_loss:.4f}, Perceptual Loss: {avg_perceptual_loss:.4f}")
        
        writer.add_scalar('Loss/train_recon', avg_recon_loss, epoch)
        writer.add_scalar('Loss/train_perceptual', avg_perceptual_loss, epoch)
        
        # --- Validace ---
        model.eval()
        total_val_recon_loss = 0
        total_val_perceptual_loss = 0
        
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch['input'].to(device)
                targets = batch['target'].to(device)
                masks = batch['mask'].to(device)
                
                outputs = model(inputs, masks)
                val_recon_loss = mse_loss(outputs, targets)
                val_p_loss = perceptual_loss(outputs, targets)
                loss_val = val_recon_loss + perceptual_weight * val_p_loss
                
                total_val_recon_loss += val_recon_loss.item()
                total_val_perceptual_loss += val_p_loss.item()
        
        avg_val_recon_loss = total_val_recon_loss / len(val_loader)
        avg_val_perceptual_loss = total_val_perceptual_loss / len(val_loader)
        
        writer.add_scalar('Loss/val_recon', avg_val_recon_loss, epoch)
        writer.add_scalar('Loss/val_perceptual', avg_val_perceptual_loss, epoch)
        print(f"Validation - Recon Loss: {avg_val_recon_loss:.4f}, Perceptual Loss: {avg_val_perceptual_loss:.4f}")
        
        # Uložení checkpointu každých 5 epoch nebo poslední epochy
        if (epoch + 1) % 5 == 0 or epoch == args.num_epochs - 1:
            checkpoint_path = os.path.join(args.output_dir, f'model_epoch{epoch+1}.pt')
            torch.save(model.state_dict(), checkpoint_path)
    
    # Uložení finálního modelu
    final_path = os.path.join(args.output_dir, 'final_model.pt')
    torch.save(model.state_dict(), final_path)
    print(f'Training completed. Final model saved to {final_path}')
    
    writer.close()

def main():
    parser = argparse.ArgumentParser(description='Fine-tune brain2vec for HIE lesion inpainting')
    parser.add_argument('--checkpoint_path', type=str, required=True, help='Path to pretrained brain2vec model')
    parser.add_argument('--csv_path', type=str, required=True, help='Path to CSV file with data')
    parser.add_argument('--output_dir', type=str, default='./output', help='Output directory')
    parser.add_argument('--batch_size', type=int, default=2, help='Batch size')
    parser.add_argument('--n_epochs', type=int, default=10, help='Number of epochs')
    parser.add_argument('--lr', type=float, default=1e-4, help='Learning rate')
    
    args = parser.parse_args()
    train(args)

if __name__ == '__main__':
    main()

In [ ]:
%cd /kaggle/working/brain2vec

!python /kaggle/working/brain2vec/preprocess_data.py

# Spuštění fine-tuningu
!python finetune_hie_lesions.py \
  --checkpoint_path ./autoencoder_final.pth \
  --csv_path ./inputs.csv \
  --output_dir /kaggle/working/hie_output \
  --batch_size 1 \
  --n_epochs 20